In [ ]:
# import packages
import ee
ee.Initialize()

import rendvi
from rendvi import eeCollections
import datetime

reNDVI supports the processing of both MODIS and VIIRS. The processing is more or less the same, the only difference is how the masking is applied. The follow code blocks provide the preprocessing for MODIS and VIIRS, so comment the block of the product to omit.

In [ ]:
# preprocessing for MODIS
mod = eeCollections.MOD09GQ['imageCollection']
mod1km = eeCollections.MOD09GA['imageCollection']

masked = rendvi.Masking.applyModis(mod,mod1km)
withNdvi = rendvi.Utils.addNDBand(
    masked,
    b1=eeCollections.MOD09GQ['nir'],
    b2=eeCollections.MOD09GQ['red'],
    outName='ndvi'
)

In [ ]:
# preprocessing for VIIRS
viirs = eeCollections.VNP09GA['imageCollection']

masked = rendvi.Masking.applyViirs(viirs)
withNdvi = rendvi.Utils.addNDBand(
    masked,
    b1=eeCollections.VNP09GA['nir'],
    b2=eeCollections.VNP09GA['red'],
    outName='ndvi'
)

In [ ]:
# time information to handle image collection
iniYear = 2000
endYear = 2019

# convert start and end dates to EE date objects
eeIni = ee.Date.fromYMD(iniYear,1,1,)
eeEnd = ee.Date.fromYMD(endYear,12,31)

# make list of years for loop processing
years = ee.List.sequence(iniYear,endYear)

In [ ]:
# instantiate a rendvi data processing object
# this has special methods for processing the data
full = rendvi.Rendvi(withNdvi,'ndvi')

In [ ]:
# calculate the dekad imagery from the full collection
dekads = full.getDekadImages(includeQa=False)

In [ ]:
# calulcate the climatology statistics for each dekad
climo = dekads.calcClimatology()

In [ ]:
# read in a land/water mask to only include ndvi from land masses
landMask = ee.Image("users/kelmarkert/public/landMask").select("land")
climo = climo.map(lambda img: img.updateMask(landMask))

In [ ]:
# change the path to theimage collection for the climatology imagery
exportAsset = "projects/servir-e-sa/reNDVI_climatology"

In [ ]:
# change the bounding box coordinates to the region for export
# coordinates should be W,S,E,N
# the following is the bounding box for Kenya
bbox = [33,-5,42,6]

In [ ]:
# change the data contact to the individual responsible for 
# managing the data
data_contact = "username@nasa.gov"

In [ ]:
# change the name of the data name prefix (optional)
data_prefix = "MOD_NDVI_CLIMATOLOGY"

In [ ]:
# change to the version/release number of interest (optional)
version_number = 0

In [ ]:
# create metadata and pyramiding policies for the data
metadataDict = dict(contact=data_contact,scale=0.0001,offset=0,version=version_number)
pyramidingDict = dict(ndvi_mean="mean",ndvi_std="mean",count="mode")
exportRegion = ee.Geometry.Rectangle(bbox,'epsg:4326',False)

In [ ]:
rendvi.batchExport(climo, 
                   exportRegion, 
                   exportAsset, 
                   prefix=data_prefix, 
                   suffix=f"v{version_number}", 
                   scale=250, 
                   crs='EPSG:4326',
                   metadata=metadataDict, 
                   pyramiding=pyramidingDict
                  )